# Assignment 2
# CS152
# Jose Nieves Flores Maynez
<br>
<br>
## DPLL Implementation

In [0]:
from copy import deepcopy

class Literal:
    """ Represents an atom or its negation depending on the sign.
    
    Arguments:
        name(str): Name of the atom.
        sign(bool): Determines if it is the atom or its negation.
    """
    def __init__(self,name, sign = True):
        self.name = name
        self.sign = sign
        
    def __neg__(self):
        # Create another instance so both can coexist
        return Literal(self.name, not self.sign)
        
    def __eq__(self,other):
        return(self.name == other.name)
        
    def __hash__(self):
        return hash(self.name)
    
    def __str__(self):
        if self.sign == False:
            print("-",self.name,end="",sep="")
        else:
            print(self.name,end="",sep="")
            
        

        
def DPLLSatisfiable(KB):
    """ Determine if a KB is satisfiable. 
    
    Arguments:
        KB (list): Contains a Knowledge Base in CNF form. It is
                   a list of sets where each set is a disjunction
                   of the literals contained with the negative
                   sign denoting the negation of a literal.
                       
    Output:
        satisfiable (bool): True if satisfiable. False otherwise.
        model (dict): Has a Literal as the key. The values can be
                      the strings "True", "False" or "Free".
    """
    
    true_lit = Literal("True") # Added to a clause to mark it True
     
    symbols = set() # Contains all symbols in KB
    
    for clause in KB:
        for literal in clause:
            symbols.add(literal)
    
    def DPLL(clauses,symbols,model):
        """ Perform DPLL algorithm to find solution.
        
        Arguments:
            clauses(list): Contains the Knowledge Base. Every element
                           in the list is a set which is a clause as 
                           a disjunction of literals.
            symbols(list): Contains all the literals appearing in the KB.
            model(dict): Contains values (bool) assigned to each symbol in 
                         the model at this stage.
                     
        Output:
            satisfiable(bool): True if satisfiable.
            model(dict): Contains values (str) assigned to each symbol in the model.
                         It is the solution. None if no solution is found.
        """
        for clause in clauses:
            counter = 0
            limit = len(clause)
            for lit in clause:
                if lit.name in model:
                    counter+=1
                    if lit.sign == model[lit.name]:
                        # Test if sign of literal matches the sign of the model.
                        # If they match, the clause is True.
                        # All literals had a default of True.
                        clause.add(true_lit)
                        # Go to next clause as this one is True.
                        break
                    elif counter == limit:
                        # Every literal is already in the model and all returned
                        # False. Must try a different model.
                        return False, None
                        
        if all([true_lit in clause for clause in clauses]):
            return True, model
        
        while symbols:
            p=symbols.pop()
            
            t_model = deepcopy(model)
            f_model = deepcopy(model)
            
            t_model[p.name] = True
            f_model[p.name] = False

            satisfiable, fin_model = DPLL(deepcopy(clauses),
                                          deepcopy(symbols),
                                          t_model)
            if satisfiable:
                return satisfiable, fin_model
            
            satisfiable, fin_model = DPLL(deepcopy(clauses),
                                          deepcopy(symbols),
                                          f_model)            
            if satisfiable:
                return satisfiable, fin_model
        
        # This only happens after trying all symbols.    
        return False, None


    satisfiable, fin_model = DPLL(KB,symbols,{})
    
    if satisfiable == True:
        model = dict(zip(fin_model.keys(), [str(v) for v in fin_model.values()]))
        for lit in symbols:
            if lit.name not in model:
                model[lit.name] = "Free"
        
    return satisfiable,model

In [0]:
def printKB(KB):
    """Prints the KB in CNF form.
    
    Arguments:
        KB(list): Contains sets that represent the clauses of the KB.
    """
    
    counter = 0
    for clause in KB:
        
        if counter!=0:
            print("∧",end="")
        
        print("(",end="")
        lit_counter = 0
        
        for lit in clause:
            
            if lit_counter!=0:
                print("∨",end="",sep="")
            
            lit.__str__()
            lit_counter+=1
        print(")",end="")
        counter+=1

printKB(KB)

(A∨B)∧(A∨-C)∧(-A∨B∨D)

## Testing

In [0]:
A = Literal("A")
B = Literal("B")
C = Literal("C")
D = Literal("D")

KB = [{A,B},{-C,A},{-A,B,D}]

print(DPLLSatisfiable(KB))

(True, {'A': 'True', 'B': 'True', 'D': 'Free', 'C': 'Free'})


In [0]:
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB2 = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, 
      {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

print(DPLLSatisfiable(KB2))


(True, {'F': 'True', 'B': 'False', 'E': 'False', 'A': 'False', 'D': 'Free', 'C': 'Free'})


# Extension

In [0]:
def DPLLSatisfiableExtended(KB):
    """ Determine if a KB is satisfiable. 
    
    Arguments:
        KB (list): Contains a Knowledge Base in CNF form. It is
                   a list of sets where each set is a disjunction
                   of the literals contained with the negative
                   sign denoting the negation of a literal.
                       
    Output:
        satisfiable (bool): True if satisfiable. False otherwise.
        model (dict): Has a Literal as the key. The values can be
                      the strings "True", "False" or "Free".
    """
    
    true_lit = Literal("True") # Added to a clause to mark it True
    
    symbol_freq = {} # Checks for frequency of symbols 
    
    symbols = set() # Contains all symbols in KB
    
    for clause in KB:
        for literal in clause:
            if literal.name not in symbol_freq:
                symbol_freq[literal.name] = 1
                symbols.add(literal)
            else:
                symbol_freq[literal.name] += 1
                
                
    def unit_clause_heur(clauses):
        """Find unit clause.
        """
        for clause in clauses:
            if len(clause) == 1:
                for lit in clause:
                    return lit
        return None
        
    def pure_symbol_heur(clauses,symbols):
        """Finds pure symbol. It appears with the same sign where it's present.
        """
        def check_pureness(clauses,symbols,symbol):
            sign = None
            for clause in clauses:
                if symbol in clause:
                    for lit in clause:
                        if lit == symbol:
                            if sign == None:
                                sign = lit.sign
                            else:
                                if sign != lit.sign:
                                    return False, None
            return True, sign
        
        for symbol in symbols:
            # Check purity for every symbol
            purity, sign = check_pureness(clauses,symbols,symbol)
            
            # Finds a pure symbol and returns it with the sign for it to be True.
            if purity == True:
                return symbol, sign
        
        # This happens when no pure symbol is found
        return None, None
                                
                            
    def DPLL(clauses,symbols,model):
        """ Perform DPLL algorithm to find solution.
        
        Arguments:
            clauses(list): Contains the Knowledge Base. Every element
                           in the list is a set which is a clause as 
                           a disjunction of literals.
            symbols(list): Contains all the literals appearing in the KB.
            model(dict): Contains values (bool) assigned to each symbol in 
                         the model at this stage.
                     
        Output:
            satisfiable(bool): True if satisfiable.
            model(dict): Contains values (str) assigned to each symbol in the model.
                         It is the solution. None if no solution is found.
        """
        for clause in clauses:
            counter = 0
            removal= set()
            limit = len(clause)
            for lit in clause:
                if lit.name in model:
                    counter +=1 # Only add +1 if literal is in model.
                    if lit.sign == model[lit.name]:
                        # Test if sign of literal matches the sign of the model.
                        # If they match, the clause is True.
                        # All literals had a default of True.
                        clause.add(true_lit)
                        # Go to next clause as this one is True.
                        break
                    if counter != limit:
                        # Can't remove while iterating so it's removed at end.
                        removal.add(lit)
                    else:
                        # Every literal is already in the model and all returned
                        # False. Must try a different model.
                        return False, None
            
            # Remove False symbols from clause.
            clause.difference_update(removal)
        if all([true_lit in clause for clause in clauses]):
            return True, model
        
        while symbols:
            
            # Apply heuristics first
            
            symbol, sign = pure_symbol_heur(clauses,symbols)
            if symbol is not None:
                # Symbol has been assigned a value so it doesn't need to
                # be popped later on
                model[symbol.name] = sign
                symbols.remove(symbol)
                return DPLL(clauses,symbols,model)
                    
            # This doesn't need a loop as it calls DPLL again inside and will
            # keep trying this heuristic first.
            unit = unit_clause_heur(clauses)
            if unit is not None:
                # The value assigned is the one of its sign so that it remains
                # True or that it becomes True by cancelling two Falses.
                model[unit.name]=unit.sign
                # The literal has already been assigned a value so it's removed.
                symbols.remove(unit)
                return DPLL(clauses,symbols,model)
            
            p=symbols.pop()
            
            t_model = deepcopy(model)
            f_model = deepcopy(model)
            
            t_model[p.name] = True
            f_model[p.name] = False

            satisfiable, fin_model = DPLL(deepcopy(clauses),
                                          deepcopy(symbols),
                                          t_model)
            if satisfiable:
                return satisfiable, fin_model
            
            satisfiable, fin_model = DPLL(deepcopy(clauses),
                                          deepcopy(symbols),
                                          f_model)            
            if satisfiable:
                return satisfiable, fin_model
        
        # This only happens after trying all symbols.    
        return False, None
#-------------------------------------------------------------------------------

    satisfiable, fin_model = DPLL(KB,symbols,{})
    

    if satisfiable == True:
        model = dict(zip(fin_model.keys(), [str(v) for v in fin_model.values()]))
        for lit in symbols:
            if lit.name not in model:
                model[lit.name] = "Free"
        
    return satisfiable,model 

# Test for Extension

In [145]:
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB2 = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, 
      {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

print(DPLLSatisfiableExtended(KB2))


(True, {'D': 'True', 'F': 'True', 'E': 'False', 'A': 'False', 'B': 'False', 'C': 'Free'})


References:
<br>
https://github.com/jcwleo/DPLL-Algorithm/blob/master/DPLL.py
<br>
<br>
Available in:
<br>